# Ray Tune - An end-to-end example of using XGBoost with Ray Tune

© 2019-2022, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

<img src="https://raw.githubusercontent.com/dmlc/dmlc.github.io/master/img/logo-m/xgboost.png" width="40%" height="20%" aligh="center">

XGBoost is currently one of the most popular machine learning algorithms for regression and classification. It performs very well on a large selection of tasks, and is the key to success in many Kaggle competitions.

Derived maily from [documentaton](https://docs.ray.io/en/latest/tune/tutorials/tune-xgboost.html), this tutorial will give you a quick introduction to XGBoost, show you how to train an XGBoost model, and then guide you on how to optimize XGBoost parameters using Ray Tune to get the best performance. In particular, we will cover the following:

 * What is XGBoost
 * Training a simple XGBoost classifier
 * XGBoost Hyperparameters
 * Tuning the configuration parameters
 * Early stopping
 * Conclusion
 * Further References

### What is XGBoost
XGBoost is an acronym for eXtreme Gradient Boosting. Internally, XGBoost uses decision trees. Instead of training just one large decision tree, XGBoost and other related algorithms train many small decision trees. The intuition behind this is that even though single decision trees can be inaccurate and suffer from high variance, combining the output of a large number of these weak learners can actually lead to strong learner, resulting in better predictions and less variance.

<img src="https://docs.ray.io/en/latest/_images/tune-xgboost-ensemble.svg" width="70%" height="50%"> 

A single decision tree (left) might be able to get to an accuracy of 70% for a binary classification task. By combining the output of several small decision trees, an ensemble learner (right) might end up with a higher accuracy of 90%.¶

Boosting algorithms start with a single small decision tree and evaluate how well it predicts the given examples. When building the next tree, those samples that have been misclassified before have a higher chance of being used to generate the tree. This is useful because it avoids overfitting to samples that can be easily classified and instead tries to come up with models that are able to classify hard examples, too. Please [see here](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205) for a more thorough introduction to bagging and boosting algorithms.

There are many boosting algorithms. In their core, they are all very similar. XGBoost uses second-level derivatives to find splits that maximize the **gain** (the inverse of the **loss**) - hence the name. In practice, there really is no drawback in using XGBoost over other boosting algorithms - in fact, it usually shows the best performance.

### Training a simple XGBoost classifier

Let’s first see how a simple XGBoost classifier can be trained. We’ll use the `breast_cancer` dataset included in the sklearn dataset collection. This is a `binary classification` dataset. Given 30 different input features, our task is to learn to identify subjects with breast cancer and those without.

In [ ]:
# !pip install -U xgboost


In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

Let's define our standard or regular XGBoost trainer (function). It takes in XGBoost configuration parameters.

In [3]:
def train_breast_cancer(config):
    # Load dataset
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
    # Split into train and test set
    train_x, test_x, train_y, test_y = train_test_split(
        data, labels, test_size=0.25)
    # Build input DMatrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set  = xgb.DMatrix(test_x, label=test_y)
    # Train the classifier
    results = {}
    bst = xgb.train(
        config,
        train_set,
        evals=[(test_set, "eval")],
        evals_result=results,
        verbose_eval=False)
    return results

Define our configurations for XGBoost

In [4]:
configs = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"]
}

In [5]:
results = train_breast_cancer(config=configs)
accuracy = 1. - results["eval"]["error"][-1]
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9510


As you can see, the code is quite simple. First, the dataset is loaded and split into a test and train set. The XGBoost model is trained with `xgb.train()`. XGBoost automatically evaluates metrics we specified on the test set. In our case it calculates the `logloss` and the prediction error, which is the percentage of misclassified examples. To calculate the accuracy, we just have to subtract the error from 1.0. Even in this simple example, most runs result in a good accuracy of over 0.90.

What if you want further accuracy, or want to use XGBoost's addiotional parameters?

### XGBoost Hyperparameters

Even with the default settings, XGBoost was able to get to a good accuracy on the breast cancer dataset. However, as in many machine learning algorithms, there are many knobs to tune which might lead to even better performance. Let’s explore some of them below.

#### Maximum tree depth
Remember that XGBoost internally uses many decision tree models to come up with predictions. When training a decision tree, we need to tell the algorithm how large the tree may get. The parameter for this is called the tree depth.

<img src="https://docs.ray.io/en/latest/_images/tune-xgboost-depth.svg" width="30%" height="10%">

In this image, the left tree has a depth of 2, and the right tree a depth of 3. Note that with each level, 2(𝑑−1) splits are added, where d is the depth of the tree.¶

Tree depth is a property that concerns the model complexity. If you only allow short trees, the models are likely not very precise - they underfit the data. If you allow very large trees, the single models are likely to overfit to the data. In practice, a number between 2 and 6 is often a good starting point for this parameter.

XGBoost’s default value is 3.

#### Minimum child weight
When a decision tree creates new leaves, it splits up the remaining data at one node into two groups. If there are only few samples in one of these groups, it often doesn’t make sense to split it further. One of the reasons for this is that the model is harder to train when we have fewer samples.

<img src="https://docs.ray.io/en/latest/_images/tune-xgboost-weight.svg" width="20%" height="10%">

In this example, we start with 100 examples. At the first node, they are split into 4 and 96 samples, respectively. In the next step, our model might find that it doesn’t make sense to split the 4 examples more. It thus only continues to add leaves on the right side.

The parameter used by the model to decide if it makes sense to split a node is called the minimum child weight. In the case of linear regression, this is just the absolute number of nodes requried in each child. In other objectives, this value is determined using the weights of the examples, hence the name.

The larger the value, the more constrained the trees are and the less deep they will be. This parameter thus also affects the model complexity. Values can range between 0 and infinity and are dependent on the sample size. For our ca. 500 examples in the breast cancer dataset, values between 0 and 10 should be sensible.

XGBoost’s default value is 1.

#### Subsample size
Each decision tree we add is trained on a subsample of the total training dataset. The probabilities for the samples are weighted according to the XGBoost algorithm, but we can decide on which fraction of the samples we want to train each decision tree on.

Setting this value to 0.7 would mean that we randomly sample 70% of the training dataset before each training iteration.

XGBoost’s default value is 1.

#### Learning rate / Eta
Remember that XGBoost sequentially trains many decision trees, and that later trees are more likely trained on data that has been misclassified by prior trees. In effect this means that earlier trees make decisions for easy samples (i.e. those samples that can easily be classified) and later trees make decisions for harder samples. It is then sensible to assume that the later trees are less accurate than earlier trees.

To address this fact, XGBoost uses a parameter called Eta, which is sometimes called the learning rate. Don’t confuse this with learning rates from gradient descent!

Typical values for this parameter are between `0.01 and 0.3`.

XGBoost’s default value is 0.3.

#### Number of boost rounds
Lastly, we can decide on how many boosting rounds we perform, which means how many decision trees we ultimately train. When we do heavy subsampling or use small learning rate, it might make sense to increase the number of boosting rounds.

XGBoost’s default value is 10.

### Putting it together

Let’s see how this looks like in code! We just need to adjust our config dict.

In [6]:
config = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "max_depth": 2,
    "min_child_weight": 0,
    "subsample": 0.8,
    "eta": 0.2
}

In [7]:
results = train_breast_cancer(config)
accuracy = 1. - results["eval"]["error"][-1]
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9161


**Note**: The accuracy is slightly lower than the default parameters used above because we randomly chose the parameters.

What if we want to get the best combination of all the parameters? This is where tuning hyperparameters helps.

### Tuning the configuration parameters
XGBoosts default parameters already lead to a good accuracy, and even our guesses in the last section should result in accuracies well above 90%. However, our guesses were just that: guesses. Often we do not know what combination of parameters would actually lead to the best results on a machine learning task.

Unfortunately, there are infinitely many combinations of hyperparameters we could try out. Should we combine `max_depth=3` with `subsample=0.8` or with `subsample=0.9?` What about the other parameters?

This is where hyperparameter tuning comes into play. By using tuning libraries such as Ray Tune, we can try out combinations of hyperparameters. Using sophisticated search strategies, these parameters can be selected so that they are likely to lead to good results (avoiding an expensive exhaustive search). Also, trials that do not perform well can be preemptively stopped to reduce waste of computing resources. 

Lastly, Ray Tune also takes care of training these runs in parallel, greatly increasing search speed.

Let’s start with a basic example on how to use Tune for this. We just need to make a few changes to our code-block:

In [8]:
from ray import tune

Add tune report to our XGBoost training function

In [9]:
def train_breast_cancer(config, checkpoint_dir=None):
     # Load dataset
     data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
     # Split into train and test set
     train_x, test_x, train_y, test_y = train_test_split(
         data, labels, test_size=0.25)
     # Build input matrices for XGBoost
     train_set = xgb.DMatrix(train_x, label=train_y)
     test_set = xgb.DMatrix(test_x, label=test_y)
     # Train the classifier
     results = {}
     xgb.train(
         config,
         train_set,
         evals=[(test_set, "eval")],
         evals_result=results,
         verbose_eval=False)
     # Return prediction accuracy
     accuracy = 1. - results["eval"]["error"][-1]
     #Below causes Ray error, solution is to pass a dictionary instead
     # tune.report(mean_accuracy=accuracy, done=True)
     tune.report({"mean_accuracy" : accuracy})

### Define our Hyperparameter Search Space

In [10]:
config = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "max_depth": tune.randint(1, 9),
    "min_child_weight": tune.choice([1, 2, 3]),
    "subsample": tune.uniform(0.5, 1.0),
    "eta": tune.loguniform(1e-4, 1e-1)
}

### Use Ray Tune parallelize our Hyperparameters tuning

This is automatically launch a Ray cluster on your laptop and schedule tasks. The `num_samples=10` option we pass to tune.run() means that we sample 10 different hyperparameter configurations from this search space, run across 10 CPUs

In [11]:
analysis = tune.run(train_breast_cancer,
         resources_per_trial={"cpu": 10},
         config=config,
         metric="mean_accuracy",                    
         mode="min",
         verbose=1,
         num_samples=10,
                    storage_path="C:\\Users\\gyanr\\gyan-python-workspace\\bayesian-optimization\\ray_run_results")

print("Done tune run")

2026-01-18 14:52:56,972	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\gyanr\AppData\Local\Temp\ray\session_2026-01-18_14-52-50_121072_11716\artifacts\2026-01-18_14-52-56\train_breast_cancer_2026-01-18_14-52-56\driver_artifacts\train_breast_cancer_48e72_00000_0_eta=0.0013,max_depth=2,min_child_weight=1,subsample=0.5777_2026-01-18_14-52-56
2026-01-18 14:52:56,988	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\gyanr\AppData\Local\Temp\ray\session_2026-01-18_14-52-50_121072_11716\artifacts\2026-01-18_14-52-56\train_breast_cancer_2026-01-18_14-52-56\driver_artifacts\train_breast_cancer_48e72_00000_0_eta=0.0013,max_depth=2,min_child_weight=1,subsample=0.5777_2026-01-18_14-52-56
2026-01-18 14:52:57,004	WARNING trial.py:647 -- The path to the trial log director

2026-01-18 14:53:02,176	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\gyanr\AppData\Local\Temp\ray\session_2026-01-18_14-52-50_121072_11716\artifacts\2026-01-18_14-52-56\train_breast_cancer_2026-01-18_14-52-56\driver_artifacts\train_breast_cancer_48e72_00000_0_eta=0.0013,max_depth=2,min_child_weight=1,subsample=0.5777_2026-01-18_14-52-56
2026-01-18 14:53:02,176	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\gyanr\AppData\Local\Temp\ray\session_2026-01-18_14-52-50_121072_11716\artifacts\2026-01-18_14-52-56\train_breast_cancer_2026-01-18_14-52-56\driver_artifacts\train_breast_cancer_48e72_00000_0_eta=0.0013,max_depth=2,min_child_weight=1,subsample=0.5777_2026-01-18_14-52-56
2026-01-18 14:53:02,208	WARNING trial.py:647 -- The path to the trial log director

Done tune run


In [12]:
print("Best Hyperparamter config: ", analysis.get_best_config(metric="mean_accuracy", mode="min"))

Best Hyperparamter config:  {'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'max_depth': 4, 'min_child_weight': 2, 'subsample': 0.5326667907021994, 'eta': 0.0001397266988818941}


### Early stopping

Currently, in our example above, Tune samples 10 different hyperparameter configurations and trains a full XGBoost on all of them. In our small example, training is very fast. However, if training were done on a large datasetm it would
take much longer and a significant amount of computer resources would be spent on trials that would eventually show a bad performance, e.g., a low accuracy. It would be good if we could identify these trials early and stop them, so we don’t waste any resources.

This is where Tune’s Schedulers shine. A Tune `TrialScheduler` is responsible for starting and stopping trials. Tune implements a number of different schedulers, each described in the Tune documentation. For our example, we will use the `AsyncHyperBandScheduler` or `ASHAScheduler`.

The basic idea of this scheduler is simple. We sample a number of hyperparameter configurations. Each of these configurations is trained for a specific number of iterations. After these iterations, only the best performing hyperparameters are retained. These are selected according to some loss metric, usually an evaluation loss. This cycle is repeated until we end up with the best configuration.

The `ASHAScheduler` needs to know three things:

 * Which metric should be used to identify badly performing trials?

 * Should this metric be maximized or minimized?

 * How many iterations does each trial train for?

There are more parameters, which are explained in the [documentation](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers).

Lastly, we have to report the loss metric to Tune. We do this with a Callback that XGBoost accepts and calls after each evaluation round. Ray Tune comes with [two XGBoost callbacks](https://docs.ray.io/en/latest/tune/api_docs/integration.html#tune-integration-xgboost) we can use for this. The `TuneReportCallback` just reports the evaluation metrics back to Tune. The `TuneReportCheckpointCallback` also saves checkpoints after each evaluation round. We will just use the latter in this example so that we can retrieve the saved model later.

These parameters from the `eval_metrics` configuration setting are then automatically reported to Tune via the callback. Here, the raw error will be reported, not the accuracy. To display the best reached accuracy, we will inverse it later.

We will also load the best checkpointed model so that we can use it for predictions. The best model is selected with respect to the `metric` and `mode` parameters we pass to `tune.run()`.

In [13]:
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback

Let's modify our training function and add our callbacks

In [14]:
def train_breast_cancer(config: dict):
    # This is a simple training function to be passed into Tune
    # Load dataset
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
    # Split into train and test set
    train_x, test_x, train_y, test_y = train_test_split(
        data, labels, test_size=0.25)
    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)
    # Train the classifier, using the Tune callback
    xgb.train(
        config,
        train_set,
        evals=[(test_set, "eval")],
        verbose_eval=False,
        callbacks=[TuneReportCheckpointCallback(filename="C:\\Users\\gyanr\\gyan-python-workspace\\bayesian-optimization\\ray_run_results\\model.xgb")])

Write a helper function for loading callbacks and returning the best model with best configuration
after tuning

In [15]:
import os
import xgboost as xgb

def get_best_model_checkpoint(analysis):
    # 1. Get the best trial object
    best_trial = analysis.get_best_trial(metric="eval-error", mode="min")

    # 2. Get the logdir (the main folder for this trial)
    trial_logdir = best_trial.local_path
    print(f"Searching for model in: {trial_logdir}")

    # 3. Search all subdirectories for 'model.xgb'
    # This bypasses the need for .checkpoint or .checkpoint_path
    model_path = None
    for root, dirs, files in os.walk(trial_logdir):
        if "model.xgb" in files:
            model_path = os.path.join(root, "model.xgb")
            break

    if not model_path:
        raise FileNotFoundError(
            f"Could not find 'model.xgb' inside {trial_logdir}. "
            "Make sure your TuneReportCheckpointCallback(filename='model.xgb') is in your training function."
        )

    # 4. Load the model
    print(f"Found and loading model from: {model_path}")
    best_bst = xgb.Booster()
    best_bst.load_model(model_path)

    # 5. Get accuracy
    accuracy = 1.0 - best_trial.last_result["eval-error"]
    print(f"Best Accuracy: {accuracy:.4f}")

    return best_bst

Wrapper around our trainer to do actual tuning:
 * define search space
 * define our ASHAScheduler
 * run tune.run(...)
 * return the ExperimentAnalysis object from `tune.run()`

In [16]:
def tune_xgboost():
    search_space = {
        # You can mix constants with search space objects.
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "max_depth": tune.randint(1, 9),
        "min_child_weight": tune.choice([1, 2, 3]),
        "subsample": tune.uniform(0.5, 1.0),
        "eta": tune.loguniform(1e-4, 1e-1)
    }
    # This will enable aggressive early stopping of bad trials.
    scheduler = ASHAScheduler(
        max_t=10,  # 10 training iterations
        grace_period=1,
        reduction_factor=2)

    analysis = tune.run(
        train_breast_cancer,   # our training function
        metric="eval-logloss", # eval metric
        mode="min",            # mode 
        # You can add "gpu": 0.1 to allocate GPUs
        resources_per_trial={"cpu": 1},
        config=search_space,
        num_samples=10,
        verbose=1,
        storage_path="C:\\Users\\gyanr\\gyan-python-workspace\\bayesian-optimization\\ray_run_results",
        scheduler=scheduler)

    return analysis

print("Done")

Done


Let's tune with our ASHAScheduler

In [17]:
analysis = tune_xgboost()
print("Done")

2026-01-18 14:54:05,225	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\gyanr\AppData\Local\Temp\ray\session_2026-01-18_14-52-50_121072_11716\artifacts\2026-01-18_14-54-05\train_breast_cancer_2026-01-18_14-54-05\driver_artifacts\train_breast_cancer_71a5c_00000_0_eta=0.0787,max_depth=6,min_child_weight=2,subsample=0.6050_2026-01-18_14-54-05
2026-01-18 14:54:05,225	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\gyanr\AppData\Local\Temp\ray\session_2026-01-18_14-52-50_121072_11716\artifacts\2026-01-18_14-54-05\train_breast_cancer_2026-01-18_14-54-05\driver_artifacts\train_breast_cancer_71a5c_00000_0_eta=0.0787,max_depth=6,min_child_weight=2,subsample=0.6050_2026-01-18_14-54-05
2026-01-18 14:54:05,225	WARNING trial.py:647 -- The path to the trial log director

2026-01-18 14:54:14,227	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\gyanr\AppData\Local\Temp\ray\session_2026-01-18_14-52-50_121072_11716\artifacts\2026-01-18_14-54-05\train_breast_cancer_2026-01-18_14-54-05\driver_artifacts\train_breast_cancer_71a5c_00004_4_eta=0.0001,max_depth=3,min_child_weight=1,subsample=0.5950_2026-01-18_14-54-05
2026-01-18 14:54:14,229	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\gyanr\AppData\Local\Temp\ray\session_2026-01-18_14-52-50_121072_11716\artifacts\2026-01-18_14-54-05\train_breast_cancer_2026-01-18_14-54-05\driver_artifacts\train_breast_cancer_71a5c_00004_4_eta=0.0001,max_depth=3,min_child_weight=1,subsample=0.5950_2026-01-18_14-54-05
2026-01-18 14:54:14,259	WARNING trial.py:647 -- The path to the trial log director

Done


In [18]:
print("Best Hyperparamter config: ", analysis.get_best_config(metric="eval-logloss", mode="min"))

Best Hyperparamter config:  {'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.605007420210472, 'eta': 0.07865221280468163}


As you can see, most trials have been stopped only after a few iterations. Only the two most promising trials were run for the full 10 iterations.

You can also ensure that all available resources are being used as the scheduler terminates trials, freeing them up. This can be done through the `ResourceChangingScheduler`. An example of this can be found here: [xgboost_dynamic_resources_example](https://docs.ray.io/en/latest/tune/examples/xgboost_dynamic_resources_example.html).



In [19]:
best_bst = get_best_model_checkpoint(analysis)

Searching for model in: C:/Users/gyanr/AppData/Local/Temp/ray/session_2026-01-18_14-52-50_121072_11716/artifacts/2026-01-18_14-54-05/train_breast_cancer_2026-01-18_14-54-05/driver_artifacts/train_breast_cancer_71a5c_00000_0_eta=0.0787,max_depth=6,min_child_weight=2,subsample=0.6050_2026-01-18_14-54-05


FileNotFoundError: Could not find 'model.xgb' inside C:/Users/gyanr/AppData/Local/Temp/ray/session_2026-01-18_14-52-50_121072_11716/artifacts/2026-01-18_14-54-05/train_breast_cancer_2026-01-18_14-54-05/driver_artifacts/train_breast_cancer_71a5c_00000_0_eta=0.0787,max_depth=6,min_child_weight=2,subsample=0.6050_2026-01-18_14-54-05. Make sure your TuneReportCheckpointCallback(filename='model.xgb') is in your training function.

In [20]:
data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
# Split into train and test set
train_x, test_x, train_y, test_y = train_test_split(
        data, labels, test_size=0.25)
test_set = xgb.DMatrix(test_x, label=test_y)

In [21]:
best_bst.predict(test_set)[:-1]

NameError: name 'best_bst' is not defined

### Conclusion
You should now have a basic understanding on how to train XGBoost models and on how to tune the hyperparameters to yield the best results. In our simple example, Tuning the parameters didn’t make a huge difference for the accuracy. But in larger applications, intelligent hyperparameter tuning can make the difference between a model that doesn’t seem to learn at all, and a model that outperforms all the other ones.



### Further References

1. [XGBoost Hyperparameter Tuning - A Visual Guide](https://kevinvecmanis.io/machine%20learning/hyperparameter%20tuning/dataviz/python/2019/05/11/XGBoost-Tuning-Visual-Guide.html)

2. [Notes on XGBoost Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

3. [Doing XGBoost Hyperparameter Tuning the smart way](https://towardsdatascience.com/doing-xgboost-hyper-parameter-tuning-the-smart-way-part-1-of-2-f6d255a45dde)
4. [Three ways to speed up XGBoost model training](https://www.anyscale.com/blog/three-ways-to-speed-up-xgboost-model-training)